## 基于词元的效果评估
最直接的效果评估是，根据每一个词元的标注结果和预测结果进行比对来计算 F1值。下面是来自《知识图谱：认知智能理论与实战》清单3-14的计算F1分数的程序：

In [1]:
from sklearn.metrics import f1_score
def evaluate_tags(fname_groundtruth, fname_test, avg='micro'):
    '''fname_groundtruth: 标注的样本；
    fname_test: 测试结果
    两个文件的格式与训练语料的格式一致'''
    with open(fname_groundtruth) as f:
        gt = [i.split('\t')[-1] for i in f.read().split('\n') ]
    with open(fname_test) as f:
        preds = [i.split('\t')[-1] for i in f.read().split('\n')]
    return f1_score(gt, preds, average=avg)


将前述的 test.txt 作为fname_groundtruth的参数，test-result.txt作为fname_test的参数，运行evaluate_tags函数即可得到F1分数。


In [2]:
evaluate_tags('test.txt', 'test-result.txt')

0.9784599304719275

默认情况下，evaluate_tags计算微观 F1分数，计算宏观（macro） F1分数 的话，则使用：

In [3]:
evaluate_tags('test.txt', 'test-result.txt', 'macro')

0.8970103545053194

## 基于实体的效果评估

<i>在实际应用中，我们通常并不关心每一个词元的标记，特别是不关心不属于任何实体类型的“O”标记，而关心抽取出来的对应于每一个实体类型的实体列表。这很好理解，比如人名“苏轼”（<苏，B-PER>, <轼，I-PER>，<在，O>）标记成“苏轼在”（ <苏，B-PER>, <轼，I-PER>，<在，I-PER>），合理的准确率是0，而不是2/3。</i>

                   ——王文广 《知识图谱：认知智能理论与实战》 P113


在实际应用中，为了更好地评估模型效果，需要对比标注的实体和模型所抽取出的实体的异同。这首先要从模型预测的结果中提取出所抽取的实体列表，《知识图谱：认知智能理论与实战》清单3-13（P111~112）示例了提取实体列表的程序，如下：


In [4]:
def get_entities(fname):
    '''适用于BIO标记方法'''
    entities = {}
    tokens_of_entity = []
    type_of_entity = None
    with open(fname) as f:
        for line in f:
            line = line.strip()
            if not line:  # 空行、句子或段落分割
                continue
            # token, label = line.split('\t')
            a = line.split('\t')
            token, label = a[0], a[-1]
            if label == 'O':  # 不属于任何实体的词元
                continue
            if label.startswith('B'):
                # print(tokens_of_entity, type_of_entity)
                if tokens_of_entity:
                    if type_of_entity in entities:
                        entities[type_of_entity].append(
                             ''.join(tokens_of_entity))
                    else:
                        entities[type_of_entity] = [
                             ''.join(tokens_of_entity)]
                tokens_of_entity = [token]
                # B-type, 比如B-ORG表示ORG类型
                type_of_entity = label[2:]
                continue
            if label.startswith('I'):
                # I-type, 比如I-ORG表示ORG类型
                assert label[2:] == type_of_entity
                tokens_of_entity.append(token)
    return entities


接着，以《知识图谱：认知智能理论与实战》清单3-15（P113）所示例的代码来计算所抽取的实体的宏观 F1分数:

In [5]:
def evaluate_entities(gt, preds):
    '''计算根据类别加权的宏观F1分数'''
    f1s = []
    for cate in gt.keys():
        y = set(gt[cate])
        y_hat = set(preds[cate])
        y_i = y.intersection(y_hat)
        p, r, f1 = 0, 0, 0
        if y_i:
            p = len(y_i) / len(y)
            r = len(y_i) / len(y_hat)
            f1 = 2 * (p * r) / (p + r)
        f1s.append(f1)
    return sum(f1s) / len(f1s)


万事俱备，接下来可以计算按照实体类型进行平均的宏观 F1分数了。此外，通常还可以对每一种类型来计算 F1分数。在此之前，由于 CRF++是将预测的标签附在标注的标签之后，为了能够用get_entities来提取实体，需要把相应的列提取出来，可用如下 shell 命令来实现：

In [6]:
!cut -d ' ' -f 1,3 test-result.txt  > preds.txt


然后，通过下面代码即可计算按实体类型来评估的宏观 F1分数:

In [7]:
# 按类型获取标注的实体
gt = get_entities('test.txt')
# 按类型获取预测的实体
preds = get_entities('preds.txt')
# 计算实体类型evaluate_entities(gt, preds)
evaluate_entities(gt, preds)

0.6987135585806094

也可以参考evaluate_entities函数的实现，计算每一个实体类型的抽取效果：

In [8]:
for cate in gt.keys():
    y = set(gt[cate])
    y_hat = set(preds[cate])
    y_i = y.intersection(y_hat)
    p, r, f1 = 0, 0, 0
    if y_i:
        p = len(y_i) / len(y)
        r = len(y_i) / len(y_hat)
        f1 = 2 * (p * r) / (p + r)
    print(cate, f1)

ORG 0.7078972407231209
PER 0.6439135381114903
LOC 0.7443298969072165


## 参考

- 王文广. 知识图谱：认知智能理论与实战[M]. 北京:电子工业出版社, 2022: P88~91,P111~113
- scikit-learn（sklearn）中的f1_score函数 的文档： https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
- 本文代码及数据可从 https://github.com/wgwang/kg-book 上获取
